# 2.1 Explicit Intent

W tym przykładzie przyjrzymy się mechanizmowi intentów, rozpoczniemy od intentów jawnych. Stworzymy aplikację z dwiema aktywnościami i umożliwimy otworzenie drugiej aktywności z poziomu pierwszej (głównej). 

Przedźmy do aktywności głównej i dodajmy przycisk do layoutu.

### Layout i utworzenie drugiej aktywności

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <Button
        android:id="@+id/button"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Button"
        android:layout_margin="16dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        android:onClick="startSecondActivity"/>
</androidx.constraintlayout.widget.ConstraintLayout>
```

Po naciśnięciu tego przycisku możliwe będzie przejście do drugiej aktywności.

Następnie dodajmy drugą aktywność do projektu. W oknie **Project** wybieram z menu kontekstowego **New -> Activity -> Empty Activity**. Zaznaczam **Generate layout file** i odznaczam **Launcher Activity**. Przejdźmy do pliku `AndroidManifest.xml`

Mamy opisane dwie aktywności. Chcemy teraz wskazać rodzica dla **SecondActivity** . Do tagu `<activity />` dodaję opcję `android:parentActivityName=".MainActivity"` oraz `android:label="Second Activity"`.

```xml
<activity
    android:name=".SecondActivity"
    android:parentActivityName=".MainActivity"
    android:label="Second Activity"
    android:exported="false" />
```

Czyli wskazaliśmy ze aktywnością nadrzędną jest **MainActivity**. Zwróćmy również uwagę na kropkę przed nazwą aktywności głównej - pominięta została nazwa pakietu (`pl.edu.uwr.pum.explicitintentkotlin.MainActivity`). `label` posłuży jako wyświetlana nazwa aktywności.

Teraz utworzymy prosty layout dla drugiej aktywności - przechodzimy do pliku `activity_second.xml` i dodajemy pole `TextView`.

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".SecondActivity">

    <TextView
        android:id="@+id/textView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Second Activity"
        android:textSize="40sp"
        android:textStyle="bold"
        android:textColor="@color/black"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />
</androidx.constraintlayout.widget.ConstraintLayout>
```

### Utworzenie `Intent` i przejście do drugiej aktywności

Dodajmy teraz obsługę przycisku w naszej głównej aktywności, przyjdźmy do pliku `MainActivity.kt`. Po wciśnięciu przycisku chcemy przejść do drugiej aktywności.

```kotlin
fun startSecondActivity(view: View) {
    
}
```

W pierwszym kroku musimy utworzyć **intent**.

```kotlin
val intent = Intent(this, SecondActivity::class.java)
```

Konstruktor klasy `Intent` przyjmuje dwa argumenty. Pierwszym jest **context**, ponieważ aktywność jest swoim własnym kontekstem używamy słowa kluczowego `this`. Drugim argumentem jest klasa aktywności którą chcemy włączyć - tutaj będzie to `SecondActivity::class.java`.

Aby włączyć aktywność wykonujemy metodę `startActivity`, jako argumentb podajemy poprzednio utworzony `Intent`.

```kotlin
startActivity(intent)
```

Na tym etapie możemy zbudować nasz projekt i przetestować funkcjonalność przycisku. Po przejściu do drugiej aktywności zwróćmy uwagę na przycisk powrotu znajdujący się na belce po lewej stronie.

<img src="images/explicitIntent1.png" width="200"/>

### Przekazanie danych do drugiej aktywności

Kolejnym krokiem będzie przekazanie danych i ich odbiór w drugiej aktywności - tutaj przekażemy tekst z pola `EditText`. Do layoutu głównej aktywności dodajmy odpowiednie pole.

```xml
<EditText
    android:id="@+id/editTextMain"
    android:layout_width="0dp"
    android:layout_height="wrap_content"
    android:layout_margin="16dp"
    android:hint="message"
    android:textStyle="bold"
    android:textSize="24sp"
    app:layout_constraintBottom_toBottomOf="parent"
    app:layout_constraintEnd_toStartOf="@+id/button"
    app:layout_constraintStart_toStartOf="parent"
    android:importantForAutofill="no"
    android:inputType="" />
```

Pierwszą rzeczą jakiej będziemy potrzebować jest to **klucz** naszej wiadomości. Przejdźmy do klasy głównej aktyności `MainActivity.kt` i dodajmy odpowiednie pole. Standardowo klucze przechowuje się w niemutowalnych polach `const` najwyższego poziomu.

```kotlin
const val EXTRA_MESSAGE = "pl.edu.uwr.pum.explicitintentkotlin.MESSAGE"
```

Standardowo nazwę stałej przechowującej klucz piszemy wielkimi literami, sam `String` zawierający klucz składa się z pakietu w której znajduje się aktywność wysyłająca i nazwy.

Dalej utwórzmy pole reprezentujące `EditText` i połączmy z elementem layoutu o odpowiednim `id`.

```kotlin
private val editText: EditText by lazy{findViewById(R.id.editTextMain)}
```

Ostatnim krokiem będzie spakowanie danych i przesłanie ich do drugiej aktywności. Przechodzimy do metody `startSecondActivity` (czyli metody `onClick` naszego przycisku) i uzupełniamy implementację.

W pierwszym kroku wyciągamy tekst z pola `EditText`

```kotlin
val message = editText.text.toString()
```

Następnie dodajemy do intentu nasze dane - czyli `String` wyciągnięty z `EditText`. W tym celu na naszym obiekcie `Intent` wywołujemy metodę `putExtra`, która dodaje dane do obiektu `Intent` i przekazuje je do aktywności odbierającej. Zrobimy to za pomocą **Scope Function** `apply`.

```kotlin
val intent = Intent(this, SecondActivity::class.java).apply {
    putExtra(EXTRA_MESSAGE, message)
}
```

Metoda przyjmuje dwa argumenty. Pierwszym jest nasz unikalny klucz, który mamy zapisany w stałej `EXTRA_MESSAGE`, drugim jest `String` który chcemy przekazać.

```kotlin
fun startSecondActivity(view: View) {
    val message = editText.text.toString()
    val intent = Intent(this, SecondActivity::class.java).apply {
        putExtra(EXTRA_MESSAGE, message)
    }
    startActivity(intent)
}
```

### Odebranie danych z poziomu drugiej aktywności

Kolejnym krokiem będzie odebranie danych z poziomu drugiej aktywności - odebrany `String` przekażemy do pola `TextView`. Odbieramy przesłane dane za pomocą metody `getStringExtra` wywołanej na odebranej instancji klasy `Intent` - w odróżnieniu od Javy nie ma konieczności jawnego wywołania metody `getIntent`. Pole `intent` umieszczone w metodzie `onCreate` zawiera instancję klasy `Intent` przekazaną przez `MainActivity`.

```kotlin
val message = intent.getStringExtra(EXTRA_MESSAGE)
```

Metoda jako argument przyjmuje unikalny klucz, który mamy zapisany w stałej w pliku `MainActivity.kt`. W ostatnim kroku chcemy odebrany `String` przekazać do pola `TextView`.

```kotlin
package pl.edu.uwr.pum.explicitintentkotlin

import androidx.appcompat.app.AppCompatActivity
import android.os.Bundle
import android.widget.TextView

class SecondActivity : AppCompatActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_second)

        val message = intent.getStringExtra(EXTRA_MESSAGE)
        findViewById<TextView>(R.id.textView).apply {
            text = message
        }
    }
}
```

Zwróćmy uwagę na wywołanie metody `findViewById`, ponieważ nie będzie nam potrzebne pole reprezentujące `TextView`, więc możemy bezpośrednia przez wywołanie tej metody dostać się po odpowiednim `id` do elementu layoutu który nas interesuje - konieczne jest jawne podanie typu `<TextView>` i za pomocą **Scope Function** ustawiamy text naszego `TextView` na `String` przekazany przez `Intent`.

Na tym etapie możemy przetestować funkcjonalność aplikacji.

### Przekazanie odpowiedzi do aktywności głównej

Kontynuujemy aplikację, kolejnym krokiem będzie zarządanie odpowiedzi z drugiej zktywności. Rozpocznijmy od zmodyfikowania layoutu drugiej aktywności - dodajmy pole `EditText` w którym wpiszemy naszą odpowiedź oraz `Button` w celu powrotu i zainicjowania powrotnego przesłania danych.

```xml
    <EditText
        android:id="@+id/editTextSecond"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:layout_margin="16dp"
        android:hint="Back message"
        android:textStyle="bold"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toStartOf="@+id/returnButton"
        app:layout_constraintStart_toStartOf="parent" />

    <Button
        android:id="@+id/returnButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Return"
        android:layout_margin="16dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        android:onClick="returnMessage"/>
```

Przejdźmy do klasy `SecondActivity.kt` i dodajmy pole przechowujące unikalny klucz naszej odpowiedzi.

```kotlin
const val EXTRA_REPLY = "pl.edu.uwr.pum.explicitintentkotlin.REPLY"
```

Następnie dodajmy metodę `onClick` przycisku **return**.

```kotlin
fun returnMessage(view: View) {
    setResult(
        RESULT_OK, 
        Intent().apply {
            putExtra(EXTRA_REPLY, 
                 findViewById<EditText>(R.id.editTextSecond).text)
    })
    finish()
}
```

Tworzymy `Intent` i wykorzystujemy metodę `putExtra` do dołączenia danych, które wyciągamy z `EditText`. Wywołujemy metodę `setResult` która przyjjmuje dwa argumenty. Pierwwzym jest kod - tutaj wykorzystamy kod `RESULT_OK`, czyli nie przewidujemy żadnych problemów z odczytaniem, jako drugi argument podajemy nasz `Intent`.

W metodzie możemy podać różne kody, które są predefiniowane w klasie `Activity` - `RESULT_OK`, `RESULT_CANCELED` i `RESULT_FIRST_USER`. Pierwszy dotyczy przypadku w których wszystko poszło po naszej myśli i możemy przekazać dane powrotne. Dzięki `RESULT_CANCELED` możemy sprecyzować co nasza aktywność główna ma zrobić gdy nie uda się wysłać danych z drugiej aktywności. Trzeci przypadek istnieje aby uniknąć konfliktów - pozwala developerowi na wykorzystanie własnego (nie predefiniowanego) kodu.

Na koniec wywołujemy metodę `finish` która kończyb działanie `SecondActivity`.

Następnie do layoutu `MainActivity` dodajmy pole `TextView` w którym wyświetlimy odebraną wiadomość.

```xml
    <TextView
        android:id="@+id/textViewMain"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Main Activity"
        android:textSize="40sp"
        android:textStyle="bold"
        android:textColor="@color/black"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />
```

Ostatnim krokiem będzie modyfikacja klasy `MainActivity.kt` w celu wysłania żądania i odebrania danych powrotnych.

W pierwszym kroku w metodzie `onClick` chcemy poozbyć się wywołania `startActivity`. Teraz nie chcemy w prosty sposób włączać innej aktywności, tylko wysłać do niej żądanie przekazania danych.

```kotlin
fun startSecondActivity(view: View) {
    val message = editText.text.toString()
    val intent = Intent(this, SecondActivity::class.java).apply {
        putExtra(EXTRA_MESSAGE, message)
    }
    //startActivity(intent)
}
```

Musimy napisać launcher naszej drugiej aktywności.

```kotlin
private val secondActivityResultLauncher = registerForActivityResult(
        ActivityResultContracts.StartActivityForResult()){
            result ->
                if (result.resultCode == Activity.RESULT_OK)
                    findViewById<TextView>(R.id.textViewMain).text =
                        result.data!!.getStringExtra(EXTRA_REPLY)
}
```

Tworzymy nowy `ActivityResultLauncher` i wywołujęmy `registerForActivityResult` który przyjmuje dwa argumenty. 

Rejestrujemy prośbę o rozpoczęcie innej aktywności. Powoduje to utworzenie wpisu w rejestrze powiązanym z drugą aktywnością, zarządzającym kodem żądania a także konwersjami do/z `Intent`. 

Jako pierwszy parametr podajemy `Contract`, nas interesuje metoda `StartActivityForResult`, poprzednio włączaliśmy drugą aktywność metodą `startActivity` - teraz żądamy odpowiedzi więc konieczne jest wykorzystanie innej meetody. Drugim argumentem jest `Callback` czyli tutaj jest to dostarczenie implementacji metody `onActivityResult` - tutaj określamy co robimy z danymi które dostajemy powrotnie z aktywności `SecondActivity`. Dla uproszczenia można to wywołanie zastąpić wyrażeniem **lambda**.

`result` jest otrzymanym z drugiej aktywności `Intent` zawierającym dane. Następnie dodajjmy implementację metody `onActivityResult`

W pierwszym kroku sprawdzam czy aktywność zwraca z poprawnym kodem

```kotlin
if (result.resultCode == Activity.RESULT_OK)
```

Następnie sprawdzamy czy dane istnieją i wyciągamy dane z `result` i umieszczamy w polu `textView`.

```kotlin
findViewById<TextView>(R.id.textViewMain).text =
                result.data!!.getStringExtra(EXTRA_REPLY)
```

Ostatnim krokiem jest wykorzystanie `secondActivityResultLauncher` w metodzie `onClick` przycisku wysyłającego żądanie

```kotlin
    fun startSecondActivity(view: View) {
        val message = editText.text.toString()
        val intent = Intent(this, SecondActivity::class.java).apply {
            putExtra(EXTRA_MESSAGE, message)
        }

        secondActivityResultLauncher.launch(intent)
    }
```

Możemy przetestowac aplikację.